In [3]:
"""
여기는 pdf to image . 

'''import os
file_list = os.listdir("./source/")
print(file_list)
'''
'''
import PyPDF2

from pdf2image import convert_from_path 
for file_name in file_list:
    pages = convert_from_path("./source/" + file_name) 

    for i, page in enumerate(pages): 
        page.save("./source/"+file_name+str(i)+".jpg", "JPEGfrom PIL import Image")
'''
"""

import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract'

from PIL import Image

#순서 문제 지문 완벽화 
def type_order(text, answer):
    first_english = False
    C_chunk = False
    temp_str = ""
    alphabet = ''
    paren = ''
    chunk = []
    perfect_text = ""
    index = 0
    
    # 문제에서 시작 문장, A,B,C 지문 chunk 추출하여 chunk에 저장. 
    while index < len(text):
        if ord('a') <= ord(text[index].lower()) <= ord('z'):
            first_english = True
        if first_english:
            if text[index] == '(':
                alphabet = text[index+1]
                paren = text[index+2]
                if alphabet == 'A' and paren == ')':
                    print(temp_str)
                    chunk.append(temp_str)
                    temp_str = ""
                    index += 3
                    continue
                elif(alphabet == 'B' and paren == ')'):
                    chunk.append(temp_str)
                    temp_str = ""
                    index += 3
                    continue
                elif(alphabet == 'C' and paren == ')'):
                    chunk.append(temp_str)
                    temp_str = ""
                    index += 3
                    C_chunk = True
                    continue
            elif text[index] == '\n':
                if(C_chunk):
                    if(text[index+1] == '\n'):
                        break
                index += 1
                continue
            temp_str += text[index]
            first_english = True
        index += 1
    chunk.append(temp_str)
    
    #시작 문장 insert
    pefect_text = chunk[0]
    #A-C-B
    if answer == 1:
        perfect_text += chunk[1]
        perfect_text += chunk[3]
        perfect_text += chunk[2]
    #B-A-C
    elif answer == 2:
        perfect_text += chunk[2]
        perfect_text += chunk[1]
        perfect_text += chunk[3]
    #B-C-A
    elif answer == 3:
        perfect_text += chunk[2]
        perfect_text += chunk[3]
        perfect_text += chunk[1]
    #C-A-B
    elif answer == 4:
        perfect_text += chunk[3]
        perfect_text += chunk[1]
        perfect_text += chunk[2]
    #C-B-A
    elif answer == 5:
        perfect_text += chunk[3]
        perfect_text += chunk[2]
        perfect_text += chunk[1]
        
    return perfect_text


#삽입 문제 지문 완벽화
def type_insert(text, answer):
    index = 0; ans_match = 0
    perfect_text = ""; to_insert = ""
    
    print("-******************************************************\n")
    #삽입할 문장 시작점 찾기
    for i in range(len(text)):
        start_flag = True
        if text[i].encode().isalpha() == False:
            continue
        else:
            for j in range(1,10):
                if (ord('가') <= ord(text[j+i]) <= ord('힣')):
                    start_flag = False
                    break
                else:
                    continue
            if start_flag == True:
                break
    #삽입할 문장 저장, 본문 시작점 찾기  
    for j in range(i,len(text)):
        if text[j] != '\n':
            to_insert += text[j]
            continue
        elif text[j] == '\n':
            if text[j+1] == '\n':
                for k in range(j+2,len(text)):
                    if text[k] != '\n':
                        break
                break
            else:
                continue   
    #print(to_insert)
    #삽입해서 완성하기
    cnt = 1
    text = text[k:]
    tmp_text = ""
    
    for i in range(len(text)):
        if i > len(text):
            print(i, len(text))
            break
        if text[i] == '(':
            for j in range(i,i+10):
                if text[j] ==  ')':
                    if cnt == answer:
                        tmp_text = text[:i] + to_insert +text[j+2:]
                        text = tmp_text
                        cnt += 1
                        break
                    else:
                        tmp_text = text[:i] + text[j+2:]
                        text = tmp_text
                        cnt += 1
                        break
                    
    print(text)
    
    return perfect_text



#빈칸 문제 지문 완벽화 
def type_blank(text, answer):
    perfect_text = []
        #삽입할 문장 시작점 찾기
    for i in range(len(text)):
        start_flag = True
        if text[i].encode().isalpha() == False:
            continue
        else:
            for j in range(1,10):
                if (ord('가') <= ord(text[j+i]) <= ord('힣')):
                    start_flag = False
                    break
                else:
                    continue
            if start_flag == True:
                break
    text = text[i:]
    idx = text.find('\n\n')
    temp_arr = text.split('\n')
    temp_arr.reverse()
    answer = list()
    ans_num = 0
    for temp in temp_arr:
        if len(temp) != 0 and temp[-1].isalpha():
            ans_num += 1
            answer.append(temp)
        if ans_num >= 5:
            break
    
    text = text.split('\n')
    text = text[:-7]
    text = " ".join(text)
    perfect_text = text[:idx] + answer[-2] + text[idx + 2:]
  

    return perfect_text
    



def scan_img(text,answer,total_img_num):
    i = 0
    for i in range(total_img_num):
        img = Image.open("input_blank"+str(i)+".png")
        one_text = pytesseract.image_to_string(img, lang='kor+eng')
        text.append(one_text)
        answer.append(2)

def scan_blank_img(text,answer,total_img_num):
    i = 0
    for i in range(total_img_num):
        img = Image.open("input_blank"+str(i)+".png")
        one_text = pytesseract.image_to_string(img, lang='kor+eng')
        text.append(one_text)
        answer.append(2)
        



#읽어들인 지문 정제. 다수의 텍스트에서 유형 판별하여 해당 함수 호출. 
def text_refinement(text, answer):
    i = 0
    blank_flag = False
    order_flag = False
    insert_flag = False
    for i in range(len(text)):
        print(text[i])
        for j in text[i] :
            #빈칸 확인
            if j == '빈':
                blank_flag = True
                continue
            if blank_flag:
                if(j == '칸'):
                    text[i] = type_blank(text[i],answer[i])
                    print(text[i])
                    break
                else:
                    blank_flag = False
            #순서 확인
            if j == '어':
                order_flag = True
                continue
            if order_flag:
                if(j == '질'):
                    text[i] = type_order(text[i],answer[i])
                    print(text[i])
                    break
                else:
                    order_flag = False
            #삽입 확인
            if (j == '흐'):
                insert_flag = True
                continue
            if insert_flag:
                if(j == '름'):
                    text[i] = type_insert(text[i],answer[i])
                    print(text[i])
                    break
                else:
                    insert_flag = False




In [6]:
#Main

text = []
answer = []
total_img_num = 1
scan_img(text,answer,total_img_num) # scan multiple image
text_refinement(text,answer)




FileNotFoundError: [Errno 2] No such file or directory: 'input_blank0.png'

In [7]:
print(text)

[]


In [32]:
scan_blank_img(text,answer,total_img_num) # scan multiple image
for num, val in enumerate(text):
    #print(num, val)
        #삽입할 문장 시작점 찾기
    for i in range(len(val)):
        start_flag = True
        if val[i].encode().isalpha() == False:
            continue
        else:
            for j in range(1,10):
                if (ord('가') <= ord(val[j+i]) <= ord('힣')):
                    start_flag = False
                    break
                else:
                    continue
            if start_flag == True:
                break
    val = val[i:]
    idx = val.find('\n\n')
    
    temp_arr = val.split('\n')
    temp_arr.reverse()
    ans = list()
    ans_num = 0
    for temp in temp_arr:
        if len(temp) != 0 and temp[-1].isalpha():
            ans_num += 1
            ans.append(temp)
        if ans_num >= 5:
            break
    print(ans)
    print(val[:idx])
    print(ans[-2])
    print(val[idx+2:])
    print('*'*100)
    

['© power shifts', '@ historical records', '® commumal responsibilities', '© personal agreements', '© religious events']
SL. In the classic model of the Sumerian economy, the temple
functioned as an administrative authority governing commodity
production, collection, and redistibution The discovery of
‘administrative tablets ftom the temple complexes at Unik
suggests that token use and consequently writing evolved as a
tool of centralized economic governance. Given the lack of
archaeological evidence from Uruk-period domestic sites, itis
not clear whether individuals also used the system for© personal agreementsFor that mutter, it isnot clear
hhow widespread literacy was at its beginnings. The use of
identifiable symbols and pictograms on the early tablets is
consistent with administrators needing 2 lexicon that was
amtually intelligible by literate and nonliterate parties. AS
ccmeiform script became more abstract, literacy must have
become increasingly important to ensure one understo

In [22]:
scan_blank_img(text,answer,total_img_num) # scan multiple image
print(text)

print('*******************************')
"""
tmp = text[1].split('\n\n')


print(tmp[1])
print('정답위치')
print(tmp[2])
print("ans:",answer)
"""

['SL. In the classic model of the Sumerian economy, the temple\nfunctioned as an administrative authority governing commodity\nproduction, collection, and redistibution The discovery of\n‘administrative tablets ftom the temple complexes at Unik\nsuggests that token use and consequently writing evolved as a\ntool of centralized economic governance. Given the lack of\narchaeological evidence from Uruk-period domestic sites, itis\nnot clear whether individuals also used the system for© personal agreementsFor that mutter, it isnot clear\nhhow widespread literacy was at its beginnings. The use of\nidentifiable symbols and pictograms on the early tablets is\nconsistent with administrators needing 2 lexicon that was\namtually intelligible by literate and nonliterate parties. AS\nccmeiform script became more abstract, literacy must have\nbecome increasingly important to ensure one understood what\nhe or she had agreed to\n\nseackacologial: 고고착적인 se esion: 141 $$\nsve cuneiform srgt 47] 문자\n© r

'\ntmp = text[1].split(\'\n\n\')\n\n\nprint(tmp[1])\nprint(\'정답위치\')\nprint(tmp[2])\nprint("ans:",answer)\n'

In [23]:
text().encode().isalpha()

TypeError: 'list' object is not callable

In [24]:
scan_img(text,answer,total_img_num) # scan multiple image
text_refinement(text,answer)

SL. In the classic model of the Sumerian economy, the temple
functioned as an administrative authority governing commodity
production, collection, and redistibution The discovery of
‘administrative tablets ftom the temple complexes at Unik
suggests that token use and consequently writing evolved as a
tool of centralized economic governance. Given the lack of
archaeological evidence from Uruk-period domestic sites, itis
not clear whether individuals also used the system for© personal agreementsFor that mutter, it isnot clear
hhow widespread literacy was at its beginnings. The use of
identifiable symbols and pictograms on the early tablets is
consistent with administrators needing 2 lexicon that was
amtually intelligible by literate and nonliterate parties. AS
ccmeiform script became more abstract, literacy must have
become increasingly important to ensure one understood what
he or she had agreed to

seackacologial: 고고착적인 se esion: 141 $$
sve cuneiform srgt 47] 문자
© religious events
© pe